In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
pune_tmax_train = pd.read_csv(r'D:\IITM\converted_to_csv\imd5p5_tmax_train_clean.csv')
Nex_train = pd.read_csv(r'D:\IITM\prepared data\Nex_train_ready.csv')

pune_tmax_test = pd.read_csv(r'D:\IITM\converted_to_csv\imd5p5_tmax_pune_test.csv')
Nex_test = pd.read_csv(r'D:\IITM\prepared data\Nex_test_ready.csv')

#using pune tmax values as labels 
tmax_labels = pd.DataFrame(pune_tmax_train,columns=["tmax"])
tmax_labels.head() # .head() function gives out first 5 rows of the datset 
nex_pr_train = pd.read_csv(r'D:\IITM\converted_to_csv\nex25_pr_train.csv')
nex_pr_test = pd.read_csv(r'D:\IITM\converted_to_csv\nex25_pr_test.csv')
Nex_train_final = pd.concat([Nex_train.reset_index(drop=True), nex_pr_train["precipitation"].reset_index(drop=True)],axis=1) 
Nex_test_final = pd.concat([Nex_test.reset_index(drop=True), nex_pr_test["precipitation"].reset_index(drop=True)],axis=1)
test_labels = pd.DataFrame(pune_tmax_test,columns=["tmax"])
test_labels.head()
nex_test_raw = pd.read_csv(r'D:\IITM\converted_to_csv\nex25_tmax_pune_test.csv')
nex_tmax = pd.DataFrame(nex_test_raw["nex_tmax"], columns=['nex_tmax'])

In [2]:
#CNN with min max scaling
from sklearn.preprocessing import MinMaxScaler
mm_X = MinMaxScaler()
mm_y = MinMaxScaler()
X_train = mm_X.fit_transform(Nex_train_final)
Y_train = mm_y.fit_transform(tmax_labels)
X_test = mm_X.fit_transform(Nex_test_final)
Y_test = mm_y.fit_transform(test_labels)

In [3]:
x_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
print(x_train.shape)
x_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
print(x_test.shape)

(10950, 4, 1)
(3650, 4, 1)


In [4]:
from keras.layers import Flatten
regressor = Sequential()
regressor.add(Flatten())
regressor.add(Dense(units=1))
regressor.compile(optimizer='adam', loss='mean_squared_error')
regressor.fit(x_train, Y_train, epochs=100, batch_size=32)

Epoch 1/100
343/343 [==============================] - 19s 809us/step - loss: 0.0215
Epoch 2/100
343/343 [==============================] - 0s 1ms/step - loss: 0.0163
Epoch 3/100
343/343 [==============================] - 0s 1ms/step - loss: 0.0143
Epoch 4/100
343/343 [==============================] - 0s 1ms/step - loss: 0.0130
Epoch 5/100
343/343 [==============================] - 0s 1ms/step - loss: 0.0129
Epoch 6/100
343/343 [==============================] - 1s 2ms/step - loss: 0.0125
Epoch 7/100
343/343 [==============================] - 1s 3ms/step - loss: 0.0122
Epoch 8/100
343/343 [==============================] - 1s 3ms/step - loss: 0.0123
Epoch 9/100
343/343 [==============================] - 1s 2ms/step - loss: 0.0125
Epoch 10/100
343/343 [==============================] - 0s 1ms/step - loss: 0.0125
Epoch 11/100
343/343 [==============================] - 0s 1ms/step - loss: 0.0122
Epoch 12/100
343/343 [==============================] - 0s 1ms/step - loss: 0.0121
Epoch 13/1

343/343 [==============================] - 1s 3ms/step - loss: 0.0125
Epoch 100/100
343/343 [==============================] - 1s 3ms/step - loss: 0.0126


In [6]:
y_test_pred_0 = regressor.predict(x_test)
cnn_test_pred_0 = mm_y.inverse_transform(y_test_pred_0)
#calculating RMSE over train set for SVR
from sklearn.metrics import mean_squared_error
cnn_mse_test = mean_squared_error(test_labels, cnn_test_pred_0)
cnn_rmse_test_0 = np.sqrt(cnn_mse_test)
print("RMSE for Support Vector Regression over test set = ")
cnn_rmse_test_0

RMSE for Support Vector Regression over test set = 


1.9262038875279834

In [25]:
from sklearn.preprocessing import Normalizer
norm_X_train = Normalizer().fit(Nex_train_final)
norm_Y_train = Normalizer().fit(tmax_labels)
norm_X_test = Normalizer().fit(Nex_test_final)
norm_Y_test = Normalizer().fit(test_labels)

In [27]:
from sklearn import preprocessing
normX_train = norm_X_train.fit_transform(Nex_train_final)
normY_train = norm_Y_train.fit_transform(tmax_labels)
normX_test = norm_X_train.fit_transform(Nex_test_final)
normY_test = norm_Y_train.fit_transform(test_labels)

In [28]:
normx_train = normX_train.reshape(normX_train.shape[0], normX_train.shape[1], 1)
print(normx_train.shape)
normx_test =normX_test.reshape(normX_test.shape[0], normX_test.shape[1], 1)
print(normx_test.shape)

(10950, 4, 1)
(3650, 4, 1)


In [29]:
from keras.layers import Flatten
regressor = Sequential()
regressor.add(Flatten())
regressor.add(Dense(units=1))
regressor.compile(optimizer='adam', loss='mean_squared_error')
regressor.fit(normx_train, normY_train, epochs=100, batch_size=32)

Epoch 1/100
343/343 [==============================] - 1s 499us/step - loss: 1.7680
Epoch 2/100
343/343 [==============================] - 0s 513us/step - loss: 0.3828
Epoch 3/100
343/343 [==============================] - 0s 546us/step - loss: 0.0570
Epoch 4/100
343/343 [==============================] - 0s 537us/step - loss: 0.0136
Epoch 5/100
343/343 [==============================] - 0s 547us/step - loss: 0.0079
Epoch 6/100
343/343 [==============================] - 0s 501us/step - loss: 0.0050
Epoch 7/100
343/343 [==============================] - 0s 490us/step - loss: 0.0030
Epoch 8/100
343/343 [==============================] - 0s 566us/step - loss: 0.0018
Epoch 9/100
343/343 [==============================] - 0s 552us/step - loss: 9.8946e-04
Epoch 10/100
343/343 [==============================] - 0s 494us/step - loss: 5.7476e-04
Epoch 11/100
343/343 [==============================] - 0s 586us/step - loss: 3.7593e-04
Epoch 12/100
343/343 [==============================] - 0s 467

343/343 [==============================] - 0s 508us/step - loss: 8.7256e-09
Epoch 94/100
343/343 [==============================] - 0s 475us/step - loss: 2.7200e-09
Epoch 95/100
343/343 [==============================] - 0s 485us/step - loss: 1.5740e-08
Epoch 96/100
343/343 [==============================] - 0s 573us/step - loss: 5.6267e-09
Epoch 97/100
343/343 [==============================] - 0s 549us/step - loss: 1.4571e-08
Epoch 98/100
343/343 [==============================] - 0s 681us/step - loss: 1.5308e-08
Epoch 99/100
343/343 [==============================] - 0s 605us/step - loss: 1.0647e-08
Epoch 100/100
343/343 [==============================] - 0s 499us/step - loss: 7.6499e-10


In [33]:
y_test_pred_0 = regressor.predict(x_test)